# 1. Linear Regression

- According to the intercept of both fitted functions, men are on average 9.76 kg (= -101.83 kg - -111.59 kg) heavyer than women, when both individuals are of equal height. 
- The slope of the height is similar, which means that the general impact of the height does not differ in both of the groups.
 

# 2. Naive Bayesian Classification

My height, weight: 175cm, 77 Kg

- The classifier would calculate two values/discriminats that must be compared in order to find out, which class would be predicted.
- The values are calculated based on Bayes theorem: $P(A|B) = \frac{P(A)*P(B|A)}{P(B)}$
- The relevant values are:
    1. $P(men|175cm \land 77Kg) = \frac{P(men)*P(175cm \land 77Kg|men)}{P(175cm \land 77Kg)}$
    2. $P(women|175cm \land 77Kg) = \frac{P(women)*P(175cm \land 77Kg|women)}{P(175cm \land 77Kg)}$
    Since the demonination in both of the formulars are equal, they are irrelevant. Thus, the formulars is transformed to:
    1. $P(men|175cm \land 77Kg) = P(men)*P(175cm \land 77Kg|men)$
    2. $P(women|175cm \land 77Kg) = P(women)*P(175cm \land 77Kg|women)$

- Then the prior probabilities $P(men)$ and $P(women)$ must be estimated. Since there are 5000 samples of each group in the dataset, $P(men)=P(women)=0.5$. Hence, they are irrelevant too.
- The probabilities of $P(175cm \land 77Kg|men)$ and $P(175cm \land 77Kg|women)$ are determined by a probability density function (PDF). This function depends on $\mu$ and $\sigma$, both given in the diagrams.
- When assuming a normal distribution in each of the predictors, then $PDF(x, \sigma, \mu) = \frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2}$


In [7]:
import math 
def pdf(x, s, m):
    return 1/(s*math.sqrt(2*math.pi))*math.pow(math.e,-0.5*((x-m)/s)**2)


print(f'P(175cm, 77Kg|men) = {pdf(175, 7.27, 175.33) * pdf(175, 8.97, 84.83) }')
print(f'P(175cm, 77Kg|women) = {pdf(175, 6.85, 161.82) * pdf(175, 8.63, 61.63) }')



P(175cm, 77Kg|men) = 2.7807942037898235e-25
P(175cm, 77Kg|women) = 1.4200526048349672e-41


Since $P(175cm \land 77Kg|men) > P(175cm \land 77Kg|women)$ in my case the classifier would predict the class men.

# 3. Naive Bayesian Classification, LDS, QDA

1. Both of the predictors grouped by the class show a normal distribution (as we already saw in the diagrams from 2.). 
2. I would suggest that the predictors for the men class come with a higher standard deviation than the predictors of the women class.

Based on the first obervation, I would expect that LDA performs better than the Naive Bayesian Classifier. The conditions for LDA are matched (normal distribution). Hence, we can get rid of the assumptions of idependen predictors, which is here definetely not the case.

Based on the second observation, I would suggest that QDA performs better than LDA. This is due to the fact that LDA assumes a similar standard deviation of the predictors independent of the response class. This is not the case in here. Additionally, QDA drops this assumption.

# 4. Resampling and trees

We make use of the OOB objects to estimate the error for each tree and then avarage them.


```
test_mse(B, N, y):
    acc_mse = 0
    for j in range(B):
        single_mse = 0
        test_data_indeces = list(filter(oob(i,j), range(N)))
        for i in test_data_indeces:
            y2 = predict(i,j)
            single_mse += mse(y[i], y2)
        acc_mse += single_mse / len(test_data_indeces)
    return acc_mse / len(B)
```


# 5. Boosting

Hyperparameters: 
- B: The amount of trees used in the ensambe
- $\lambda$: the learning rate
- d: the amount of splits considered for each tree

Reasonable lower and upper bounds:
- For B a lower bound is 2. Otherwise the algorithms would just train one tree, which has no benefit compared to an single tree. The upper bound is not limited. However, one has to consider, that the model is more complicated to understand, if there are more trees involved. Therefore, more trees would reduce one of the most important advantage of the tree based methods, i.e. that they are easy to understand.

- For $\lambda$ the lower bound is close but greater than zero. If $\lambda = 0$ the model would not evolve and thus result in the initinal setting ($f(x) = 0$). An upper bound for a learning rate is typically one. This is due to the fact, that the learning rate works like a percentual weight for the trained tree. If $\lambda > 0$ the residuals used in the next iteration would be not longer related to the original response values, e.g. with response value 1, $\lambda=5$ and a assumed residual of the first tree of 0.5  $r = 1 - 5*0.5 = - 1.5$. In the given example, the next tree would overcompensate to this error.

- For d there is a lower bound of 1, since a tree need at least one split. A upper bound is in theory just set by the amount of datapoints N at d=N-1. However, one has to consider the same question as for B, when selecting this hyperparameter.

# 6. Support Vector Machines

The data of the red class is centered around a mean of 0. The data of the blue class if located around this red class spot. Based on this, a polynomial or the radial kernel would help to add a third dimension, that lifts the datapoints according to the predictor variables X1 and X2. In the result, the blue points will be lifted much more than the red points. Hence, after the linear kernel application, a hyperplane separating these two classes can be created.